# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Image Analysis

BrainStation Winter 2023: Data Science

This notebook is for running my images through deep learning networks. The images have been pre-processed in a separate notebook.

The goal is to find a pre-trained CNN to adapt to our specific use-case of finding insights about wildfire-prone landscapes.

## Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# import `ImageDataGenerator` to help facilitate 
# loading images directly from our computer

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Specify the dimensions for the target preprocess image size

height = 128 
width = 128
channels = 3

# Create training image data generator
train_datagen = ImageDataGenerator(rescale=1./255)

# Create validation image data generator
# Only apply rescaling to our validation data
validation_datagen = ImageDataGenerator(rescale=1./255)